## Calculo de tiempo promedio de horas pico de las sucursales 
### Desarrollado por:
- **Nombre:** Gabriel Chavez 
- **Fecha:** 2024-11-21

### Descripción
Tomaremos de referencia las transacciones de kielsa Honduras, donde pondremos de forma ordenada las cajas y los id de facturas, para poder calcular el tiempo promedio de horas pico de las sucursales.
- La hora de inicio sera la fecha de finazalizacion anterior.
- El tiempo promedio sera la diferencia entre la hora de inicio y la hora de fin de la transaccion.
- Organizar por tipo de transaccion y sacar el promedio.


In [24]:
import polars as pl
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import text

from sqlalchemy import create_engine
from urllib.parse import quote_plus


connection_string = (
    'mssql+pyodbc://'
    'Angel_chavez:{}@172.16.2.227\\DWHFARINTERDEV/BI_FARINTER?'
    'driver=ODBC+Driver+17+for+SQL+Server'.format(quote_plus('@ng3l_ch@v3z'))
)
engine = create_engine(connection_string)
query = text("""SELECT
    fc.Suc_Id,
    fc.Emp_Id,
    fc.Caja_Id,
    fc.Factura_Id,
    fc.Factura_Fecha,
    fc.Factura_FechaHora,
    fc.AnioMes_Id,
    fc.Cliente_Id,
    fc.MonederoTarj_Id,
    fc.Vendedor_Id,
    CASE 
        WHEN fc.Monedero_Id = '0' THEN 'No Identificada'
        ELSE md.Tipo_Plan 
    END AS Tipo_Plan,
    md.Edad,
    suc.TipoSucursal_Id,
    suc.TipoSucursal_Nombre,
    suc.Zona_Id,
    suc.Zona_Nombre
FROM
    [BI_Kielsa_Hecho_FacturaEncabezado] AS fc
    JOIN [BI_Kielsa_Dim_Monedero] AS md 
        ON md.Monedero_Id = fc.Monedero_Id AND md.Emp_Id = fc.Emp_Id       
    JOIN [BI_Kielsa_Dim_Sucursal] AS suc 
        ON suc.Sucursal_Id = fc.Suc_Id AND suc.Emp_Id = fc.Emp_Id
WHERE
    fc.AnioMes_Id = 202410 
    AND fc.Emp_Id = 1;""")

with engine.connect() as conn:
    df_base = pd.read_sql(query, conn).set_index('Factura_Id').reset_index()



In [25]:
df_base.head()

,Factura_Id,Suc_Id,Emp_Id,Caja_Id,Factura_Fecha,Factura_FechaHora,AnioMes_Id,Cliente_Id,MonederoTarj_Id,Vendedor_Id,Tipo_Plan,Edad,TipoSucursal_Id,TipoSucursal_Nombre,Zona_Id,Zona_Nombre
0,15073,118,1,3,2024-10-26,13:20:38,202410,10000001,0502-1996-01137,10002837,Monedero todo publico,28,8,Supermercado,2,NORTE
1,119584,56,1,1,2024-10-01,13:11:58,202410,10000001,0502-1996-01177,10003784,Monedero todo publico,28,3,Clínica/Hospitalaria,2,NORTE
2,17786,268,1,1,2024-10-22,16:26:31,202410,10000001,0502-1996-01178,10004530,Monedero todo publico,28,9,Plaza Comercial,2,NORTE
3,73545,248,1,4,2024-10-22,17:19:22,202410,10000001,0502-1996-01288,10004684,Monedero todo publico,29,9,Plaza Comercial,2,NORTE
4,22382,268,1,2,2024-10-30,19:09:34,202410,10000001,0502-1996-01632,10004530,Monedero todo publico,28,9,Plaza Comercial,2,NORTE


In [30]:
import numpy as np

# Copia de DataFrame base
df = df_base.copy()

tipos_transaccion = {
    'Monedero tercera edad': 'Tercera Edad',
    'Monedero Cuarta Edad': 'Tercera Edad',
    'Monedero todo publico': 'Todo Publico',
    'Monedero Clínica TP': 'Todo Publico',
    'Monedero Clínica TE': 'Tercera Edad',
    'No Identificada': 'No Identificada',
}

# Mapear los tipos de transacciones y eliminar las no relevantes
df['Tipo_Plan'] = df['Tipo_Plan'].map(tipos_transaccion)
df = df.dropna(subset=['Tipo_Plan'])


# Convertir Factura_FechaHora a cadena para agregar fecha ficticia
df['Factura_FechaHora'] = df['Factura_FechaHora'].astype(str)
df['Factura_FechaHora'] = '1900-01-01 ' + df['Factura_FechaHora']

# Convertir Factura_FechaHora a tipo datetime
df['Factura_FechaHora'] = pd.to_datetime(df['Factura_FechaHora'], errors='coerce')

# Asegurarse de que Factura_Fecha sea tipo datetime
df['Factura_Fecha'] = pd.to_datetime(df['Factura_Fecha'])

# Ordenar el DataFrame
df = df.sort_values(by=['Suc_Id', 'Caja_Id', 'Factura_Id', 'Factura_Fecha'], ascending=[True, True, True, True])

# Crear columna Factura_Inicio_FechaHora solo si la fecha es la misma
df['Factura_Inicio_FechaHora'] = np.where(
    df['Factura_Fecha'] == df['Factura_Fecha'].shift(1),
    df['Factura_FechaHora'].shift(1),
    pd.NaT  # Asignar NaT si no coinciden
)

# Asegurarse de que la columna Factura_Inicio_FechaHora sea tipo datetime
df['Factura_Inicio_FechaHora'] = pd.to_datetime(df['Factura_Inicio_FechaHora'], errors='coerce')

# Calcular el tiempo transcurrido
df['Tiempo_Transcurrido'] = df['Factura_FechaHora'] - df['Factura_Inicio_FechaHora']

# convertir el tiempo transcurrido a minutos
df['Tiempo_Transcurrido'] = df['Tiempo_Transcurrido'].dt.total_seconds() / 60
# excluir columnas con un Factura_Inicio_FechaHora igual a NaT
df = df.dropna(subset=['Factura_Inicio_FechaHora'])



In [73]:
print(df[df.Tiempo_Transcurrido < 0])

        Factura_Id  Suc_Id  Emp_Id  Caja_Id Factura_Fecha   Factura_FechaHora  \
808655      238609      52       1        1    2024-10-11 1900-01-01 13:16:04   
113603      130435      52       1        2    2024-10-13 1900-01-01 18:22:33   
491710      225646      58       1        4    2024-10-03 1900-01-01 18:23:27   
323232      125088      93       1        1    2024-10-31 1900-01-01 08:43:12   
302829       77763     107       1        1    2024-10-18 1900-01-01 15:22:18   
444261       77905     107       1        1    2024-10-24 1900-01-01 09:33:47   
603241      159916     111       1        2    2024-10-16 1900-01-01 10:59:30   
549642       87480     124       1        2    2024-10-08 1900-01-01 12:34:28   
544480      209954     142       1        1    2024-10-08 1900-01-01 11:54:47   
629586       58402     183       1        2    2024-10-16 1900-01-01 18:17:34   
711321       55698     204       1        3    2024-10-15 1900-01-01 11:44:28   
290215       89726     209  

In [78]:
print(df[(df.Factura_Id.isin([238608, 238609])) & (df.Suc_Id == 52)])

        Factura_Id  Suc_Id  Emp_Id  Caja_Id Factura_Fecha   Factura_FechaHora  \
540715      238608      52       1        1    2024-10-11 1900-01-01 13:16:18   
808655      238609      52       1        1    2024-10-11 1900-01-01 13:16:04   

        AnioMes_Id  Cliente_Id MonederoTarj_Id  Vendedor_Id        Tipo_Plan  \
540715      202410    10000001               0     10003928  No Identificada   
808655      202410    10000001               0     10003928  No Identificada   

        Edad  TipoSucursal_Id TipoSucursal_Nombre  Zona_Id Zona_Nombre  \
540715    33                2    Centro Comercial        2       NORTE   
808655    33                2    Centro Comercial        2       NORTE   

       Factura_Inicio_FechaHora  Tiempo_Transcurrido  Hora  
540715      1900-01-01 13:13:24             2.900000    13  
808655      1900-01-01 13:16:18            -0.233333    13  


In [63]:
# Agregar una columna para la hora
df['Hora'] = df['Factura_FechaHora'].dt.hour
# Agrupar por hora y tipo (por ejemplo, Tipo_Plan) y calcular métricas
resultados = df.groupby(['Hora', 'Tipo_Plan', 'Suc_Id', 'Caja_Id', 'Factura_Fecha']).agg(
    Tiempo_Promedio=('Tiempo_Transcurrido', 'mean'),
    Cantidad_Transacciones=('Factura_Id', 'count')
).reset_index()

resultados.Tiempo_Promedio.max()

print(resultados)

        Hora        Tipo_Plan  Suc_Id  Caja_Id Factura_Fecha  Tiempo_Promedio  \
0          0  No Identificada       3        1    2024-10-04         2.940000   
1          0  No Identificada       3        1    2024-10-19         5.913889   
2          0  No Identificada       3        2    2024-10-01         6.870000   
3          0  No Identificada       3        2    2024-10-02         3.922222   
4          0  No Identificada       3        2    2024-10-03         3.740476   
...      ...              ...     ...      ...           ...              ...   
361787    23     Todo Publico     305        1    2024-10-12        12.141667   
361788    23     Todo Publico     305        1    2024-10-14        35.300000   
361789    23     Todo Publico     305        1    2024-10-16        44.216667   
361790    23     Todo Publico     305        1    2024-10-19        85.883333   
361791    23     Todo Publico     305        1    2024-10-30        43.616667   

        Cantidad_Transaccio

In [62]:
# Agrupar por Sucursal, Hora y Tipo_Plan
promedios_por_sucursal = resultados.groupby(['Suc_Id', 'Hora', 'Tipo_Plan']).agg(
    Promedio_Tiempo=('Tiempo_Promedio', 'mean'),
    Promedio_Transacciones=('Cantidad_Transacciones', 'mean')
).reset_index()

# promeedios de tiempos por tipo
promedios_por_tipo = promedios_por_sucursal.groupby([ 'Tipo_Plan']).agg(
    Promedio_Tiempo=('Promedio_Tiempo', 'mean')
).reset_index()

print(promedios_por_tipo)


         Tipo_Plan  Promedio_Tiempo
0  No Identificada        18.580983
1     Tercera Edad        20.165213
2     Todo Publico        19.801017
